# US Forest Fire Data
Dataset includes 24 years of US forest fires geospatial data from 1992-2015. 

## Look at the Big Picture

As someone passionate about trees and the forests they help create, I am interested in examining the causes, size and intensity of forest fires in the US. Given the US Forest Fires dataset found on kaggle.com, I spend some time here exploring and visualizing the data. There are a lot of other interesting analyses that I might come back to but this is a solid start.

## Get the Data

Data was downloaded as a sqlite database from Kaggle.com.

[Link to dataset](https://www.kaggle.com/rtatman/188-million-us-wildfires)

### Dataset Description from Kaggle.com
"This data publication contains a spatial database of wildfires that occurred in the United States from 1992 to 2015. It is the third update of a publication originally generated to support the national Fire Program Analysis (FPA) system. The wildfire records were acquired from the reporting systems of federal, state, and local fire organizations. The following core data elements were required for records to be included in this data publication: discovery date, final fire size, and a point location at least as precise as Public Land Survey System (PLSS) section (1-square mile grid). The data were transformed to conform, when possible, to the data standards of the National Wildfire Coordinating Group (NWCG). Basic error-checking was performed and redundant records were identified and removed, to the degree possible. The resulting product, referred to as the Fire Program Analysis fire-occurrence database (FPA FOD), includes 1.88 million geo-referenced wildfire records, representing a total of 140 million acres burned during the 24-year period."

### Acknowledgements:
These data were collected using funding from the U.S. Government and can be used without additional permissions or fees. If you use these data in a publication, presentation, or other research product please use the following citation:

Short, Karen C. 2017. Spatial wildfire occurrence data for the United States, 1992-2015 [FPAFOD20170508]. 4th Edition. Fort Collins, CO: Forest Service Research Data Archive. https://doi.org/10.2737/RDS-2013-0009.4


In [ ]:
import calplot
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

type(0x7A6CF84A)  # %matplotli 0x7A6CF84A

import plotly.express as px
from plotly import offline
from plotly.offline import init_notebook_mode

init_notebook_mode(connected=True)

import datetime as dt
import json
import sqlite3
from os import path

import requests
import topojson

In [ ]:
# global variables and settings
plt.style.use("dark_background")
plt.rcParams.update({"figure.figsize": (15, 10)})

seq_cmap = "YlOrRd"

In [ ]:
def sqlite3_connection(db):
    """ Create a sqlite3 database connection. """

    # check sqlite3 database file exists
    try:
        path.exists(db)
    except:
        print("sqlite3 database file does not exist.")
        return False

    # connect to the sqlite3 database
    conn = None
    try:

        conn = sqlite3.connect(db)
        cur = conn.cursor()
        print("sqlite3 version:", sqlite3.version)
        return conn, cur
    except:
        return sqlite3.Error

In [ ]:
conn, cur = sqlite3_connection("FPA_FOD_20170508.sqlite")

In [ ]:
def sqlite3_execute(q, cur=cur, fetchall=True):
    """ Securely execute specific sqlite3. """

    try:
        cur.execute(q)
        if fetchall:
            return cur.fetchall()
        else:
            return cur.fetchone()
    except:
        return sqlite3.Error

### Take a look at the database tables

In [ ]:
tables = sqlite3_execute("SELECT name FROM sqlite_master WHERE type='table';")
tables

In [ ]:
print(f"The number of tables in the database: {len(tables)}")

In [ ]:
q = """
    SELECT * FROM Fires
    LIMIT 10;
    """

sqlite3_execute(q, fetchall=False)

The `Fires` table appears to be the data we are looking for. The next question is, what do each of the other tables represent?

In [ ]:
count = 0
print(f"{'TABLE' : <30}{'NUMBER OF ROWS' : >10}")

for t in tables:
    q = f"""
        SELECT COUNT(*) FROM {t[0]};
        """

    try:
        shape = sqlite3_execute(q)
        nrows = shape[0][0]
        if nrows != 0:
            print(f"{f'{t[0]}':<30}{f'{nrows}':<10}")
            count += 1
    except:
        continue

print(f"\nThe number of non-empty tables: {count}")

Just about half of the tables in this sqlite database are empty or raise `sqlite3.Error` messages. 

In [ ]:
q = """
    SELECT * FROM NWCG_UnitIDActive_20170109;
    """

pd.read_sql(q, conn).head()

By simplying looking at the tables themselves (using `pd.read_sql`), the following tables seem to be of interest and may need to be merged in some fashion:
* `Fires` 
* `idx_Fires_Shape`
* `idx_Fires_Shape_node`
* `idx_Fires_Shape_rowid`
* `idx_Fires_Shape_parent`
* `NWCG_UnitIDActive_20170109`
    * "Look-up table containing all NWCG identifiers for agency units that were active (i.e., valid) as of 9 January 2017, when the list was downloaded from https://www.nifc.blm.gov/unit_id/Publish.html and used as the source of values available to populate the following fields in the Fires table: NWCGREPORTINGAGENCY, NWCGREPORTINGUNITID, and NWCGREPORTINGUNITNAME."

### Explore the `Fires` table

In [ ]:
%%time

q = """
    SELECT * FROM Fires;
    """

# `fires` is a copy of the `Fires` table
fires = pd.read_sql(q, conn)

In [ ]:
# save `fires` table to a csv
fires.to_csv("fires.csv", index=False)

In [ ]:
fires.info()

In [ ]:
fires.describe()

In [ ]:
fires.NWCG_REPORTING_AGENCY.value_counts(dropna=False)

In [ ]:
fires.FIRE_CODE.value_counts(dropna=False)

## Explore the data

In [ ]:
# `low_memory` set to False ensures that there are no mixed types
fires = pd.read_csv("fires.csv", low_memory=False)

In [ ]:
fires.info()

### Initial obeservations 
Many of the columns represent data specific to the agency in charge of fire and how those agency track the fires internally. Most of this data is of little use to us. The columns that are of most interest to us start at column 19 or `FIRE_YEAR`.

To start, let's look at the discovery date (`DISCOVERY_DATE`) and containment date (`CONT_DATE`) to confirm that the date range is between 1992 and 2015.

In [ ]:
(fires.FIRE_YEAR.value_counts(dropna=False).sort_index().plot(kind="line", x=fires))

plt.title("Number of US Forest Fires between 1992 and 2015")
plt.ylabel("Number of total fires")
plt.xlabel("Year")
plt.show();

In [ ]:
fires.DISCOVERY_DATE.head()

In [ ]:
fires.DISCOVERY_DATE.isna().sum()

In [ ]:
fires.DISCOVERY_DOY.head()

In [ ]:
fires.DISCOVERY_TIME.head()

In [ ]:
fires.DISCOVERY_TIME.isna().sum() / len(fires)

In [ ]:
fires.DISCOVERY_TIME.plot(kind="hist", bins=25)

plt.title("Discovery Times for US Forest Fires")
plt.xlabel("Time (24H clock)")
plt.show();

### Observations about when fires were "discovered"

The `DISCOVERY_DATE` are unrecognizable. However, we are in luck, this question was already answered in the [Kaggle discussion section for this dataset](https://www.kaggle.com/rtatman/188-million-us-wildfires/discussion/39627). These appear to be Julian dates and "dates are simply a continuous count of days and fractions since noon Universal Time on January 1, 4713 BC (on the Julian calendar)". This makes a lot more sense. 

As for `DISCOVERY_DOY`, they appear to represent the day of the year, making this a redudant column.

And lastly, the `DISCOVERY_TIME` appear to represent the time of the day. This distribution appears to be fairly Gaussian with a mean around 15:00 in the afternoon. However just under half of these times are missing. This begs the question, if almost half of the data are missing, is it worth including this attribute in further analysis? 

In [ ]:
fires.STAT_CAUSE_CODE.value_counts(dropna=False)

In [ ]:
fires.STAT_CAUSE_DESCR.value_counts(dropna=False)

### Observations about the "statistical" cause of the fire

There are two attributes which represent the "statistical" cause of the fire. `STAT_CAUSE_DESCR` provides a brief description of the statistcal cause and `STAT_CAUSE_CODE` encodes these descriptions. There are no missing values for either, that said two of the largest "statistical" causes are "Miscellaneous" and "Missing/Undefined". 

This attribute represent a potential target variable, or y-variable. 

In [ ]:
fires.CONT_DATE.head()

In [ ]:
fires.CONT_DATE.isna().sum() / len(fires)

In [ ]:
fires.CONT_DOY.head()

In [ ]:
fires_train.CONT_TIME.isna().sum() / len(fires_train)

In [ ]:
fires[fires.CONT_DATE.isna()].FIRE_SIZE_CLASS.value_counts(normalize=True)

In [ ]:
fires.FIRE_SIZE_CLASS.value_counts(dropna=False, normalize=True)

In [ ]:
fires.CONT_DATE.isna().sum() / len(fires)

In [ ]:
fires.CONT_TIME.median()

In [ ]:
fires.CONT_TIME.plot(kind="hist", bins=25)

plt.title("Time of Containment for US Forest Fires")
plt.xlabel("Time (24H clock)")
plt.show();

### Observations about the "containment" date

Again, the `CONT_DATE` appears to be Julian dates but unlike `DISCOVERY_DATE`, half of the values seem to be missing. Since this attribute will be useful in determining the duration of the fires, it would be useful to impute those missing values if possible.

`CONT_DOY` also seems to represent the day of the year and therefore a redudant attribute.

Lastly, `CONT_TIME` attribute is missing over 50% of its values and again, the question is whether this attribute is worth using for further analysis.

In [ ]:
fires.STAT_CAUSE_CODE.isna().sum()

In [ ]:
fires.STAT_CAUSE_DESCR.value_counts()

In [ ]:
fires.FIRE_SIZE.isna().sum()

In [ ]:
fires.FIRE_SIZE.min()

In [ ]:
fires.FIRE_SIZE.max()

In [ ]:
fires.FIRE_SIZE_CLASS.value_counts(dropna=False)

In [ ]:
fires.FIRE_SIZE_CLASS.value_counts(dropna=False).sort_index().plot(kind="bar")

plt.title("US Forest Fire Size Classes")
plt.ylabel("Frequency")
plt.xlabel("Size Class")
plt.show();

In [ ]:
fires.FIRE_SIZE.plot(kind="hist", bins=50, logy=True)

plt.title("US Forest Fire, Log Distribution of Sizes")
plt.ylabel("Frequency (log scale)")
plt.xlabel("Size of Fire (acres)")
plt.show();

### Observations about the "size" of the fire

The `FIRE_SIZE` attribute represents the "estimate of acres within the final perimeter of the fire". The data is heavily skewed to the left (smaller fires). 

From the kaggle.com description of `FIRE_SIZE_CLASS` attribute:
* A=greater than 0 but less than or equal to 0.25 acres
* B=0.26-9.9 acres
* C=10.0-99.9 acres 
* D=100-299 acres 
* E=300 to 999 acres
* F=1000 to 4999 acres
* G=5000+ acres

There are no missing values and these attributes might be able to help us impute the containment date. If the fire is small (class A or B) perhaps that's an indication that the fire was contained the same day it was discovered.

`FIRE_SIZE` is a potential target variable or y-variable for future machine learning exercise.

In [ ]:
fires.LATITUDE.head()

In [ ]:
fires.LONGITUDE.head()

In [ ]:
fires.plot(
    kind="scatter",
    x="LONGITUDE",
    y="LATITUDE",
    alpha=0.4,
    s=fires["FIRE_SIZE"] / 500,
    c=np.log(fires["FIRE_SIZE"]),
    figsize=(15, 10),
    colormap=plt.get_cmap("spring"),
)

plt.title("US Forest Fire Locations")
plt.ylabel("Latitude")
plt.xlabel("Longitude")
plt.show();

### Observations about the fire locations

We can clearly make out the continental United States, Hawaii, Alaska and Puerto Rico simply by plotting the location (and size) of the forest fires. There appears to be a greater concentration of large fires in the western half of the continental US as well as many large fires in Alaska. There is a noticeable gap in the US midwest in areas around Ohio, Indiana, Illinois, and Iowa.



In [ ]:
fires.OWNER_CODE.value_counts(dropna=False)

In [ ]:
fires.OWNER_DESCR.value_counts(dropna=False)

In [ ]:
fires_by_state_series = fires.STATE.value_counts(dropna=False)
fires_by_state = pd.DataFrame(
    {"state": fires_by_state_series.index, "total_fires": fires_by_state_series.values}
)
fires_by_state.head()

### References

* [^1] - Thanks to user `mbostock` for the topoJSON data, more info found on their [GitHub](https://github.com/topojson/us-atlas).
* [^2] - Thanks to Sean Gillies for his useful [`topojson.py` library](https://sgillies.net/2012/11/27/topojson-py.html) used to help convert topoJSON to geoJSON. [Link to topojson GitHub Repo](https://github.com/sgillies/topojson).
* [^3] - Thanks to whoever create the brief tutorial on converting topoJSON to geoJSON data found [here](https://chart-studio.plotly.com/~empet/14397.embed).
* [^4] - Thanks to user `mshafrir` for their "US state in JSON" [GitHub Gist](https://gist.github.com/mshafrir/2646763).

In [ ]:
def get_json(url):
    """ Given a url with raw json, download and return the json. """

    try:
        r = requests.get(url)
        r.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Encountered an error:\n\n{SystemExit(e)}")

    return r.json()

In [ ]:
state_dict_url = "https://gist.githubusercontent.com/mshafrir/2646763/raw/8b0dbb93521f5d6889502305335104218454c2bf/states_hash.json"
topoJSON_url = "https://cdn.jsdelivr.net/npm/us-atlas@3/counties-10m.json"

state_dict = get_json(state_dict_url)
topoJSON = get_json(topoJSON_url)

In [ ]:
def topo2geo(topoJSON, level):
    """ Convert a topoJSON object to geoJSON object for specified level `nation`, `states`, or `counties`. """

    topo_features = topoJSON["objects"][f"{level}"]["geometries"]
    scale = topoJSON["transform"]["scale"]
    translation = topoJSON["transform"]["translate"]

    geoJSON = dict(type="FeatureCollection", features=[])

    for k, tfeature in enumerate(topo_features):
        geo_feature = dict(id=k, type="Feature")
        if level != "nation":
            geo_feature["properties"] = tfeature["properties"]
        geo_feature["geometry"] = topojson.geometry(
            tfeature, topoJSON["arcs"], scale, translation
        )
        geoJSON["features"].append(geo_feature)

    return geoJSON

In [ ]:
geoJSON = topo2geo(topoJSON, "states")

In [ ]:
# grab the 'id' for each present US state or territory from geoJSON data
# unique numerical ID needed for plotly.express.choropleth
state_list_map = {}
for state in geoJSON["features"]:
    state_list_map[state["properties"]["name"]] = state["id"]


# create a mapping dict for state abbreviations and their corresponding ID
# i.e. {'CA' : 'California'} and {'California' : 21} --> {'CA' :  21}
state_id_map = {}
for sd in state_dict.items():
    for slm in state_list_map.items():
        if sd[1].lower() == slm[0].lower():
            state_id_map[sd[0]] = slm[1]

In [ ]:
fires_by_state["id"] = fires_by_state["state"].apply(lambda x: state_id_map[x])

In [ ]:
fires_by_state.head()

In [ ]:
fig = px.choropleth(
    fires_by_state,
    locations="id",
    geojson=geoJSON,
    color="total_fires",
    scope="usa",
    color_continuous_scale=px.colors.sequential.YlOrRd,
)
offline.iplot(fig, filename="fires_by_states")

### Observations about US states and their numer of total fires

Note: The data above only represents 80% of the total fires since the remaining 20% are saved in the test data set. This map also does not include forest fires that occurred in US territories, namely Puerto Rico. 

Frmo the map, there are clearly a handful of places that experience many more forest fires than other. For example, California, Texas and Georgia appear to have the greatest number of fires. However as we saw above, the vast majority of fires are small (less than 10 acres). This map does not take the number of acres a single fire burned through as such, map underrepresent some states such as Alaska, or the US northwest; this is show to be the case when we plotted the size of the fire based on their longitute and latitude, see further up.


In [ ]:
fires_by_county_series = fires.COUNTY.value_counts(dropna=False)
fires_by_county = pd.DataFrame(
    {
        "county": fires_by_county_series.index,
        "total_fires": fires_by_county_series.values,
    }
)
fires_by_county.head()

In [ ]:
fires.COUNTY.value_counts(dropna=False)

In [ ]:
fires.COUNTY.isna().sum() / len(fires)

In [ ]:
fires.FIPS_CODE.value_counts(dropna=False)

In [ ]:
fires.FIPS_NAME.value_counts(dropna=False)

In [ ]:
fires[fires["FIPS_NAME"] == "Washington"].STATE.value_counts()

### Observation about US counties, FIPS Codes and FIPS Names

There are a total of 3006 counties across the US and from the data above, we see there are 3376 counties represented. What makes up for the difference? There is a clue at the fourth to last county listed, "Humboldt / Pershing". It appears there may be a handful of forest fires that have been labelled with two (or more) county namaes. There are also appears to be integer values present ("5") and over 36% of the data is missing for this attribute. 

As much as I would like to plot forest fires data at the county level (like we did for states), I suspect this will take considerable time to untangle.

There are two FIPS columns, `FIPS_CODE` and `FIPS_NAME`, where FIPS stands for "Federal Information Processing Standard" which uniquely identifies counties or county equivalents in the US. As of 2008, they are no longer a "standard" though still in use to varying degrees. This makes this column especially difficult to us since 2008 happens to fall in the middle of the date range for our data set. 

In [ ]:
fires.Shape.head()